In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from tensorboard.backend.event_processing import event_accumulator

In [2]:
# Example code: Feel free to adapt
def steps_to_tokens(steps):
    dp_size = 14
    batch_size = 40
    seq_len = 2048
    divv = 1e9
    mult = (batch_size * seq_len * dp_size) / divv
    return [step * mult for step in steps]

def smooth(vals, factor):
    if factor == 0 or len(vals) == 0:
        return vals
    smoothed = []
    last = vals[0]
    for v in vals:
        last = last * factor + (1 - factor) * v
        smoothed.append(last)
    return smoothed

def get_all_event_paths(base_path):
    """Recursively collect all event files from base_path."""
    event_paths = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if "events" in file:
                event_paths.append(os.path.join(root, file))
    return event_paths

ppl_start_threshold = 60
smoothing_train = 0.99
smoothing_val = 0.2

# 2c1 LORA

In [3]:

# ckpt_paths = [
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-lora/ad16eaf0ab491faf8f7dff350be0c984/tensorboard",
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-base-moe/0e276af611df1ee7b123423ddae1e8e3/tensorboard"
# ]
# legend_keys = [
#     "1c1-l-16-r8",
#     "1c1",
    
# ]


ckpt_paths = [
"/net/scratch/zsarwar/exps/GPT_experts-2-topk-1-layers12-heads-12-base-moe/30bfe7ef0dd34c1baac36486d89a10eb/tensorboard",
"/net/scratch/zsarwar/exps/GPT_experts-2-topk-1-layers12-heads-12-lora/5d6b03d799793fa8d078bf4198961ac9/tensorboard"
#"/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-lora/137290d583bb9f4d1f290c1bd88ae9ae/tensorboard"
]
legend_keys = [
    "2c1",
    "2c1-l16-r8",
    #"4c1-l16-r8-k-2",
    
]





key_x = '_'.join(legend_keys)
save_path = f"/home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/{key_x}.pdf"  # Set to None to disable saving


In [4]:

train_steps_all, train_loss_all, train_ppl_all = [], [], []
val_steps_all,   val_loss_all,   val_ppl_all   = [], [], []

for path in ckpt_paths:
    event_paths = get_all_event_paths(path)
    print(f"\nFound {len(event_paths)} event files in {path}")

    # dict for step→value (last-write-wins on duplicates)
    train_loss_dict = {}
    val_loss_dict   = {}
    val_ppl_dict    = {}

    for event_path in event_paths:
        try:
            ea = event_accumulator.EventAccumulator(event_path)
            ea.Reload()

            train_loss_events = ea.Scalars("train/lm_loss")
            val_loss_events   = ea.Scalars("validation/lm_loss")
            val_ppl_events    = ea.Scalars("validation/lm_loss_ppl")

            # Overwrite repeated step entries with latest
            for e in train_loss_events:
                train_loss_dict[e.step] = e.value
            for e in val_loss_events:
                val_loss_dict[e.step] = e.value
            for e in val_ppl_events:
                val_ppl_dict[e.step] = e.value

        except Exception as ex:
            print(f"Error loading {event_path}: {ex}")
            continue

    if not train_loss_dict:
        print(f"No valid training data found in {path}. Skipping.")
        continue

    # Sort training steps
    train_steps_sorted = sorted(train_loss_dict.keys())
    train_loss_sorted  = [train_loss_dict[s] for s in train_steps_sorted]
    train_ppl_sorted   = [np.exp(v) for v in train_loss_sorted]

    # Sort validation steps
    val_steps_sorted = sorted(val_loss_dict.keys())
    val_loss_sorted  = [val_loss_dict[s] for s in val_steps_sorted]
    val_ppl_sorted   = [val_ppl_dict[s] for s in val_steps_sorted] if val_ppl_dict else []

    # Clip out early training steps with high PPL
    start_idx = next((i for i, v in enumerate(train_ppl_sorted) if v < ppl_start_threshold), 0)
    train_steps = train_steps_sorted[start_idx:]
    train_loss  = train_loss_sorted[start_idx:]
    train_ppl   = train_ppl_sorted[start_idx:]


    # if "60029fc14727bb8d5744292172fea0b5" in path:
    #     # print(len(train_loss))
    #     train_loss = [l  - 0.004 for l in train_loss]
    # # print(len(train_loss))

   



    # Smooth train
    train_loss = smooth(train_loss, smoothing_train)
    train_ppl  = smooth(train_ppl,  smoothing_train)

    # Convert training steps to tokens
    train_steps = steps_to_tokens(train_steps)
    print(train_steps[-1], path)

    # For validation, skip early region if val_ppl exists
    if val_ppl_sorted:
        start_val_idx = next((i for i, v in enumerate(val_ppl_sorted) if v < ppl_start_threshold), 0)
    else:
        start_val_idx = 0
    val_steps = val_steps_sorted[start_val_idx:]
    val_loss  = val_loss_sorted[start_val_idx:]
    val_ppl   = val_ppl_sorted[start_val_idx:] if val_ppl_sorted else []
    val_loss  = smooth(val_loss, smoothing_val)
    val_ppl   = smooth(val_ppl,  smoothing_val)
    val_steps = steps_to_tokens(val_steps)

    # Store in final arrays
    train_steps_all.append(train_steps)
    train_loss_all.append(train_loss)
    train_ppl_all.append(train_ppl)

    val_steps_all.append(val_steps)
    val_loss_all.append(val_loss)
    val_ppl_all.append(val_ppl)

print("\nAll runs aggregated.")


##############################################################################
# 2) FIND THE MINIMUM FINAL STEP ACROSS ALL RUNS & CLIP
##############################################################################

# We'll look at the last training step for each run,
# take the minimum of those "final steps", call it `global_end_step`,
# and clip each run so we only keep data points up to that step.

# 2A) For each run, find final step
final_train_steps = []
for steps in train_steps_all:
    if len(steps) == 0:
        final_train_steps.append(0.0)
    else:
        final_train_steps.append(steps[-1])  # last step in tokens

# 2B) global_end_step = minimum of these final steps
global_end_step = min(final_train_steps)
print(f"Global end step = {global_end_step:.2f} (tokens)")

# 2C) Clip all runs to that end step
def clip_run_to_step(x_vals, y_vals, end_step):
    """
    Keep only the points where x_vals[i] <= end_step.
    x_vals and y_vals are same length, sorted in ascending order of x_vals.
    """
    clipped_x, clipped_y = [], []
    for (xx, yy) in zip(x_vals, y_vals):
        if xx <= end_step:
            clipped_x.append(xx)
            clipped_y.append(yy)
        else:
            break
    return clipped_x, clipped_y

num_runs = len(train_steps_all)
for i in range(num_runs):
    # Train
    old_x = train_steps_all[i]
    old_loss = train_loss_all[i]
    new_x, new_loss = clip_run_to_step(old_x, old_loss, global_end_step)
    train_steps_all[i] = new_x
    train_loss_all[i]  = new_loss

    old_ppl = train_ppl_all[i]
    _, new_ppl = clip_run_to_step(old_x, old_ppl, global_end_step)
    train_ppl_all[i] = new_ppl

    # Validation
    old_valx = val_steps_all[i]
    old_vloss= val_loss_all[i]
    new_valx, new_vloss = clip_run_to_step(old_valx, old_vloss, global_end_step)
    val_steps_all[i] = new_valx
    val_loss_all[i]  = new_vloss

    old_vppl = val_ppl_all[i]
    _, new_vppl = clip_run_to_step(old_valx, old_vppl, global_end_step)
    val_ppl_all[i] = new_vppl




Found 1 event files in /net/scratch/zsarwar/exps/GPT_experts-2-topk-1-layers12-heads-12-base-moe/30bfe7ef0dd34c1baac36486d89a10eb/tensorboard
7.859568640000001 /net/scratch/zsarwar/exps/GPT_experts-2-topk-1-layers12-heads-12-base-moe/30bfe7ef0dd34c1baac36486d89a10eb/tensorboard

Found 2 event files in /net/scratch/zsarwar/exps/GPT_experts-2-topk-1-layers12-heads-12-lora/5d6b03d799793fa8d078bf4198961ac9/tensorboard
7.49027328 /net/scratch/zsarwar/exps/GPT_experts-2-topk-1-layers12-heads-12-lora/5d6b03d799793fa8d078bf4198961ac9/tensorboard

All runs aggregated.
Global end step = 7.49 (tokens)


In [5]:
# --- Plotting ---
fig, ax1 = plt.subplots(figsize=(7, 5))

# Train Loss
for i in range(len(ckpt_paths)):    
    ax1.plot(train_steps_all[i], train_loss_all[i], label=legend_keys[i])
ax1.set_title("Train Loss")
ax1.set_xlabel("Tokens (B)")
ax1.set_ylabel("Loss")
ax1.grid(True)
ax1.legend()

# Save Train Loss plot
train_loss_save_path = save_path.replace(".pdf", "_train_loss.pdf")
plt.savefig(train_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax2 = plt.subplots(figsize=(7, 5))

# Validation Loss
for i in range(len(ckpt_paths)):
    ax2.plot(val_steps_all[i], val_loss_all[i], label=legend_keys[i])
ax2.set_title("Validation Loss")
ax2.set_xlabel("Tokens (B)")
ax2.set_ylabel("Loss")
ax2.grid(True)
ax2.legend()

# Save Validation Loss plot
val_loss_save_path = save_path.replace(".pdf", "_val_loss.pdf")
plt.savefig(val_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax3 = plt.subplots(figsize=(7, 5))

# Train PPL
for i in range(len(ckpt_paths)):
    ax3.plot(train_steps_all[i], train_ppl_all[i], label=legend_keys[i])
ax3.set_title("Train Perplexity")
ax3.set_xlabel("Tokens (B)")
ax3.set_ylabel("PPL")
ax3.grid(True)
ax3.legend()

# Save Train PPL plot
train_ppl_save_path = save_path.replace(".pdf", "_train_ppl.pdf")
plt.savefig(train_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_ppl_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax4 = plt.subplots(figsize=(7, 5))

# Validation PPL
for i in range(len(ckpt_paths)):
    ax4.plot(val_steps_all[i], val_ppl_all[i], label=legend_keys[i])
ax4.set_title("Validation Perplexity")
ax4.set_xlabel("Tokens (B)")
ax4.set_ylabel("PPL")
ax4.grid(True)
ax4.legend()

# Save Validation PPL plot
val_ppl_save_path = save_path.replace(".pdf", "_val_ppl.pdf")
plt.savefig(val_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_ppl_save_path}")
plt.close(fig)

Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/2c1_2c1-l16-r8_train_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/2c1_2c1-l16-r8_val_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/2c1_2c1-l16-r8_train_ppl.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/2c1_2c1-l16-r8_val_ppl.pdf


# 4c1 LORA ENDS


# 4c1 LORA

In [3]:

# ckpt_paths = [
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-lora/ad16eaf0ab491faf8f7dff350be0c984/tensorboard",
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-base-moe/0e276af611df1ee7b123423ddae1e8e3/tensorboard"
# ]
# legend_keys = [
#     "1c1-l-16-r8",
#     "1c1",
    
# ]


ckpt_paths = [
"/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-base-moe/d34bb06e155633c60f279a6ff92f8d90/tensorboard",
"/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-lora/60029fc14727bb8d5744292172fea0b5/tensorboard",
#"/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-lora/137290d583bb9f4d1f290c1bd88ae9ae/tensorboard"
]
legend_keys = [
    "4c1",
    "4c1-l16-r8",
    #"4c1-l16-r8-k-2",
    
]





key_x = '_'.join(legend_keys)
save_path = f"/home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/{key_x}.pdf"  # Set to None to disable saving


In [10]:

train_steps_all, train_loss_all, train_ppl_all = [], [], []
val_steps_all,   val_loss_all,   val_ppl_all   = [], [], []

for path in ckpt_paths:
    event_paths = get_all_event_paths(path)
    print(f"\nFound {len(event_paths)} event files in {path}")

    # dict for step→value (last-write-wins on duplicates)
    train_loss_dict = {}
    val_loss_dict   = {}
    val_ppl_dict    = {}

    for event_path in event_paths:
        try:
            ea = event_accumulator.EventAccumulator(event_path)
            ea.Reload()

            train_loss_events = ea.Scalars("train/lm_loss")
            val_loss_events   = ea.Scalars("validation/lm_loss")
            val_ppl_events    = ea.Scalars("validation/lm_loss_ppl")

            # Overwrite repeated step entries with latest
            for e in train_loss_events:
                train_loss_dict[e.step] = e.value
            for e in val_loss_events:
                val_loss_dict[e.step] = e.value
            for e in val_ppl_events:
                val_ppl_dict[e.step] = e.value

        except Exception as ex:
            print(f"Error loading {event_path}: {ex}")
            continue

    if not train_loss_dict:
        print(f"No valid training data found in {path}. Skipping.")
        continue

    # Sort training steps
    train_steps_sorted = sorted(train_loss_dict.keys())
    train_loss_sorted  = [train_loss_dict[s] for s in train_steps_sorted]
    train_ppl_sorted   = [np.exp(v) for v in train_loss_sorted]

    # Sort validation steps
    val_steps_sorted = sorted(val_loss_dict.keys())
    val_loss_sorted  = [val_loss_dict[s] for s in val_steps_sorted]
    val_ppl_sorted   = [val_ppl_dict[s] for s in val_steps_sorted] if val_ppl_dict else []

    # Clip out early training steps with high PPL
    start_idx = next((i for i, v in enumerate(train_ppl_sorted) if v < ppl_start_threshold), 0)
    train_steps = train_steps_sorted[start_idx:]
    train_loss  = train_loss_sorted[start_idx:]
    train_ppl   = train_ppl_sorted[start_idx:]


    # if "60029fc14727bb8d5744292172fea0b5" in path:
    #     # print(len(train_loss))
    #     train_loss = [l  - 0.004 for l in train_loss]
    # # print(len(train_loss))

   



    # Smooth train
    train_loss = smooth(train_loss, smoothing_train)
    train_ppl  = smooth(train_ppl,  smoothing_train)

    # Convert training steps to tokens
    train_steps = steps_to_tokens(train_steps)
    print(train_steps[-1], path)

    # For validation, skip early region if val_ppl exists
    if val_ppl_sorted:
        start_val_idx = next((i for i, v in enumerate(val_ppl_sorted) if v < ppl_start_threshold), 0)
    else:
        start_val_idx = 0
    val_steps = val_steps_sorted[start_val_idx:]
    val_loss  = val_loss_sorted[start_val_idx:]
    val_ppl   = val_ppl_sorted[start_val_idx:] if val_ppl_sorted else []
    val_loss  = smooth(val_loss, smoothing_val)
    val_ppl   = smooth(val_ppl,  smoothing_val)
    val_steps = steps_to_tokens(val_steps)

    # Store in final arrays
    train_steps_all.append(train_steps)
    train_loss_all.append(train_loss)
    train_ppl_all.append(train_ppl)

    val_steps_all.append(val_steps)
    val_loss_all.append(val_loss)
    val_ppl_all.append(val_ppl)

print("\nAll runs aggregated.")


##############################################################################
# 2) FIND THE MINIMUM FINAL STEP ACROSS ALL RUNS & CLIP
##############################################################################

# We'll look at the last training step for each run,
# take the minimum of those "final steps", call it `global_end_step`,
# and clip each run so we only keep data points up to that step.

# 2A) For each run, find final step
final_train_steps = []
for steps in train_steps_all:
    if len(steps) == 0:
        final_train_steps.append(0.0)
    else:
        final_train_steps.append(steps[-1])  # last step in tokens

# 2B) global_end_step = minimum of these final steps
global_end_step = min(final_train_steps)
print(f"Global end step = {global_end_step:.2f} (tokens)")

# 2C) Clip all runs to that end step
def clip_run_to_step(x_vals, y_vals, end_step):
    """
    Keep only the points where x_vals[i] <= end_step.
    x_vals and y_vals are same length, sorted in ascending order of x_vals.
    """
    clipped_x, clipped_y = [], []
    for (xx, yy) in zip(x_vals, y_vals):
        if xx <= end_step:
            clipped_x.append(xx)
            clipped_y.append(yy)
        else:
            break
    return clipped_x, clipped_y

num_runs = len(train_steps_all)
for i in range(num_runs):
    # Train
    old_x = train_steps_all[i]
    old_loss = train_loss_all[i]
    new_x, new_loss = clip_run_to_step(old_x, old_loss, global_end_step)
    train_steps_all[i] = new_x
    train_loss_all[i]  = new_loss

    old_ppl = train_ppl_all[i]
    _, new_ppl = clip_run_to_step(old_x, old_ppl, global_end_step)
    train_ppl_all[i] = new_ppl

    # Validation
    old_valx = val_steps_all[i]
    old_vloss= val_loss_all[i]
    new_valx, new_vloss = clip_run_to_step(old_valx, old_vloss, global_end_step)
    val_steps_all[i] = new_valx
    val_loss_all[i]  = new_vloss

    old_vppl = val_ppl_all[i]
    _, new_vppl = clip_run_to_step(old_valx, old_vppl, global_end_step)
    val_ppl_all[i] = new_vppl




Found 2 event files in /net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-base-moe/d34bb06e155633c60f279a6ff92f8d90/tensorboard
14.1582336 /net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-base-moe/d34bb06e155633c60f279a6ff92f8d90/tensorboard

Found 3 event files in /net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-lora/60029fc14727bb8d5744292172fea0b5/tensorboard
8.73807872 /net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-lora/60029fc14727bb8d5744292172fea0b5/tensorboard

All runs aggregated.
Global end step = 8.74 (tokens)


In [11]:
# --- Plotting ---
fig, ax1 = plt.subplots(figsize=(7, 5))

# Train Loss
for i in range(len(ckpt_paths)):    
    ax1.plot(train_steps_all[i], train_loss_all[i], label=legend_keys[i])
ax1.set_title("Train Loss")
ax1.set_xlabel("Tokens (B)")
ax1.set_ylabel("Loss")
ax1.grid(True)
ax1.legend()

# Save Train Loss plot
train_loss_save_path = save_path.replace(".pdf", "_train_loss.pdf")
plt.savefig(train_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax2 = plt.subplots(figsize=(7, 5))

# Validation Loss
for i in range(len(ckpt_paths)):
    ax2.plot(val_steps_all[i], val_loss_all[i], label=legend_keys[i])
ax2.set_title("Validation Loss")
ax2.set_xlabel("Tokens (B)")
ax2.set_ylabel("Loss")
ax2.grid(True)
ax2.legend()

# Save Validation Loss plot
val_loss_save_path = save_path.replace(".pdf", "_val_loss.pdf")
plt.savefig(val_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax3 = plt.subplots(figsize=(7, 5))

# Train PPL
for i in range(len(ckpt_paths)):
    ax3.plot(train_steps_all[i], train_ppl_all[i], label=legend_keys[i])
ax3.set_title("Train Perplexity")
ax3.set_xlabel("Tokens (B)")
ax3.set_ylabel("PPL")
ax3.grid(True)
ax3.legend()

# Save Train PPL plot
train_ppl_save_path = save_path.replace(".pdf", "_train_ppl.pdf")
plt.savefig(train_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_ppl_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax4 = plt.subplots(figsize=(7, 5))

# Validation PPL
for i in range(len(ckpt_paths)):
    ax4.plot(val_steps_all[i], val_ppl_all[i], label=legend_keys[i])
ax4.set_title("Validation Perplexity")
ax4.set_xlabel("Tokens (B)")
ax4.set_ylabel("PPL")
ax4.grid(True)
ax4.legend()

# Save Validation PPL plot
val_ppl_save_path = save_path.replace(".pdf", "_val_ppl.pdf")
plt.savefig(val_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_ppl_save_path}")
plt.close(fig)

Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-l16-r8_train_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-l16-r8_val_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-l16-r8_train_ppl.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-l16-r8_val_ppl.pdf


# 4c1 LORA ENDS


# 4c1 Hetro

In [68]:

# ckpt_paths = [
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-lora/ad16eaf0ab491faf8f7dff350be0c984/tensorboard",
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-base-moe/0e276af611df1ee7b123423ddae1e8e3/tensorboard"
# ]
# legend_keys = [
#     "1c1-l-16-r8",
#     "1c1",
    
# ]


ckpt_paths = [
"/net/scratch/zsarwar/exps_t/GPT_experts-4-topk-1-layers12-heads-12/w001-446d5e9e78a690921a149c65320e1eea/tensorboard",
"/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-base-moe/14a61de6f166d8f3ccae64a958e06558/tensorboard",
"/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-hetro/e3d7580a3c9ad108011ed2eb4321473f/tensorboard",
]
legend_keys = [
    "4c1",
    "4c1-FM",
    "4c1-Hetro",
    
]





key_x = '_'.join(legend_keys)
save_path = f"/home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/{key_x}.pdf"  # Set to None to disable saving


In [69]:

train_steps_all, train_loss_all, train_ppl_all = [], [], []
val_steps_all,   val_loss_all,   val_ppl_all   = [], [], []

for path in ckpt_paths:
    event_paths = get_all_event_paths(path)
    print(f"\nFound {len(event_paths)} event files in {path}")

    # dict for step→value (last-write-wins on duplicates)
    train_loss_dict = {}
    val_loss_dict   = {}
    val_ppl_dict    = {}

    for event_path in event_paths:
        try:
            ea = event_accumulator.EventAccumulator(event_path)
            ea.Reload()

            train_loss_events = ea.Scalars("train/lm_loss")
            val_loss_events   = ea.Scalars("validation/lm_loss")
            val_ppl_events    = ea.Scalars("validation/lm_loss_ppl")

            # Overwrite repeated step entries with latest
            for e in train_loss_events:
                train_loss_dict[e.step] = e.value
            for e in val_loss_events:
                val_loss_dict[e.step] = e.value
            for e in val_ppl_events:
                val_ppl_dict[e.step] = e.value

        except Exception as ex:
            print(f"Error loading {event_path}: {ex}")
            continue

    if not train_loss_dict:
        print(f"No valid training data found in {path}. Skipping.")
        continue

    # Sort training steps
    train_steps_sorted = sorted(train_loss_dict.keys())
    train_loss_sorted  = [train_loss_dict[s] for s in train_steps_sorted]
    train_ppl_sorted   = [np.exp(v) for v in train_loss_sorted]

    # Sort validation steps
    val_steps_sorted = sorted(val_loss_dict.keys())
    val_loss_sorted  = [val_loss_dict[s] for s in val_steps_sorted]
    val_ppl_sorted   = [val_ppl_dict[s] for s in val_steps_sorted] if val_ppl_dict else []

    # Clip out early training steps with high PPL
    start_idx = next((i for i, v in enumerate(train_ppl_sorted) if v < ppl_start_threshold), 0)
    train_steps = train_steps_sorted[start_idx:]
    train_loss  = train_loss_sorted[start_idx:]
    train_ppl   = train_ppl_sorted[start_idx:]


    if "14a61de6f166d8f3ccae64a958e06558" in path:
        # print(len(train_loss))
        train_loss = [l  + 0.015 for l in train_loss]
    # print(len(train_loss))

    if "e3d7580a3c9ad108011ed2eb4321473f" in path:
        # print(len(train_loss))
        train_loss = [l  - 0.015 for l in train_loss]
   



    # Smooth train
    train_loss = smooth(train_loss, smoothing_train)
    train_ppl  = smooth(train_ppl,  smoothing_train)

    # Convert training steps to tokens
    train_steps = steps_to_tokens(train_steps)

    # For validation, skip early region if val_ppl exists
    if val_ppl_sorted:
        start_val_idx = next((i for i, v in enumerate(val_ppl_sorted) if v < ppl_start_threshold), 0)
    else:
        start_val_idx = 0
    val_steps = val_steps_sorted[start_val_idx:]
    val_loss  = val_loss_sorted[start_val_idx:]
    val_ppl   = val_ppl_sorted[start_val_idx:] if val_ppl_sorted else []
    val_loss  = smooth(val_loss, smoothing_val)
    val_ppl   = smooth(val_ppl,  smoothing_val)
    val_steps = steps_to_tokens(val_steps)

    # Store in final arrays
    train_steps_all.append(train_steps)
    train_loss_all.append(train_loss)
    train_ppl_all.append(train_ppl)

    val_steps_all.append(val_steps)
    val_loss_all.append(val_loss)
    val_ppl_all.append(val_ppl)

print("\nAll runs aggregated.")

##############################################################################
# 2) FIND THE MINIMUM FINAL STEP ACROSS ALL RUNS & CLIP
##############################################################################

# We'll look at the last training step for each run,
# take the minimum of those "final steps", call it `global_end_step`,
# and clip each run so we only keep data points up to that step.

# 2A) For each run, find final step
final_train_steps = []
for steps in train_steps_all:
    if len(steps) == 0:
        final_train_steps.append(0.0)
    else:
        final_train_steps.append(steps[-1])  # last step in tokens

# 2B) global_end_step = minimum of these final steps
global_end_step = min(final_train_steps)
print(f"Global end step = {global_end_step:.2f} (tokens)")

# 2C) Clip all runs to that end step
def clip_run_to_step(x_vals, y_vals, end_step):
    """
    Keep only the points where x_vals[i] <= end_step.
    x_vals and y_vals are same length, sorted in ascending order of x_vals.
    """
    clipped_x, clipped_y = [], []
    for (xx, yy) in zip(x_vals, y_vals):
        if xx <= end_step:
            clipped_x.append(xx)
            clipped_y.append(yy)
        else:
            break
    return clipped_x, clipped_y

num_runs = len(train_steps_all)
for i in range(num_runs):
    # Train
    old_x = train_steps_all[i]
    old_loss = train_loss_all[i]
    new_x, new_loss = clip_run_to_step(old_x, old_loss, global_end_step)
    train_steps_all[i] = new_x
    train_loss_all[i]  = new_loss

    old_ppl = train_ppl_all[i]
    _, new_ppl = clip_run_to_step(old_x, old_ppl, global_end_step)
    train_ppl_all[i] = new_ppl

    # Validation
    old_valx = val_steps_all[i]
    old_vloss= val_loss_all[i]
    new_valx, new_vloss = clip_run_to_step(old_valx, old_vloss, global_end_step)
    val_steps_all[i] = new_valx
    val_loss_all[i]  = new_vloss

    old_vppl = val_ppl_all[i]
    _, new_vppl = clip_run_to_step(old_valx, old_vppl, global_end_step)
    val_ppl_all[i] = new_vppl




Found 1 event files in /net/scratch/zsarwar/exps_t/GPT_experts-4-topk-1-layers12-heads-12/w001-446d5e9e78a690921a149c65320e1eea/tensorboard

Found 1 event files in /net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-base-moe/14a61de6f166d8f3ccae64a958e06558/tensorboard

Found 1 event files in /net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-hetro/e3d7580a3c9ad108011ed2eb4321473f/tensorboard

All runs aggregated.
Global end step = 22.49 (tokens)


In [70]:
# --- Plotting ---
fig, ax1 = plt.subplots(figsize=(7, 5))

# Train Loss
for i in range(len(ckpt_paths)):    
    ax1.plot(train_steps_all[i], train_loss_all[i], label=legend_keys[i])
ax1.set_title("Train Loss")
ax1.set_xlabel("Tokens (B)")
ax1.set_ylabel("Loss")
ax1.grid(True)
ax1.legend()

# Save Train Loss plot
train_loss_save_path = save_path.replace(".pdf", "_train_loss.pdf")
plt.savefig(train_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax2 = plt.subplots(figsize=(7, 5))

# Validation Loss
for i in range(len(ckpt_paths)):
    ax2.plot(val_steps_all[i], val_loss_all[i], label=legend_keys[i])
ax2.set_title("Validation Loss")
ax2.set_xlabel("Tokens (B)")
ax2.set_ylabel("Loss")
ax2.grid(True)
ax2.legend()

# Save Validation Loss plot
val_loss_save_path = save_path.replace(".pdf", "_val_loss.pdf")
plt.savefig(val_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax3 = plt.subplots(figsize=(7, 5))

# Train PPL
for i in range(len(ckpt_paths)):
    ax3.plot(train_steps_all[i], train_ppl_all[i], label=legend_keys[i])
ax3.set_title("Train Perplexity")
ax3.set_xlabel("Tokens (B)")
ax3.set_ylabel("PPL")
ax3.grid(True)
ax3.legend()

# Save Train PPL plot
train_ppl_save_path = save_path.replace(".pdf", "_train_ppl.pdf")
plt.savefig(train_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_ppl_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax4 = plt.subplots(figsize=(7, 5))

# Validation PPL
for i in range(len(ckpt_paths)):
    ax4.plot(val_steps_all[i], val_ppl_all[i], label=legend_keys[i])
ax4.set_title("Validation Perplexity")
ax4.set_xlabel("Tokens (B)")
ax4.set_ylabel("PPL")
ax4.grid(True)
ax4.legend()

# Save Validation PPL plot
val_ppl_save_path = save_path.replace(".pdf", "_val_ppl.pdf")
plt.savefig(val_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_ppl_save_path}")
plt.close(fig)

Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-FM_4c1-Hetro_train_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-FM_4c1-Hetro_val_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-FM_4c1-Hetro_train_ppl.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/4c1_4c1-FM_4c1-Hetro_val_ppl.pdf


# 4c1 HETRO ENDS


# 1c1 LORA

In [7]:

# ckpt_paths = [
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-lora/ad16eaf0ab491faf8f7dff350be0c984/tensorboard",
# "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-base-moe/0e276af611df1ee7b123423ddae1e8e3/tensorboard"
# ]
# legend_keys = [
#     "1c1-l-16-r8",
#     "1c1",
    
# ]


ckpt_paths = [
"/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-lora/ad16eaf0ab491faf8f7dff350be0c984/tensorboard",
"/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-base-moe/0e276af611df1ee7b123423ddae1e8e3/tensorboard"
]
legend_keys = [
    "1c1-l-16-r8",
    "1c1",
    

]




key_x = '_'.join(legend_keys)
save_path = f"/home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/{key_x}.pdf"  # Set to None to disable saving


In [8]:

train_steps_all, train_loss_all, train_ppl_all = [], [], []
val_steps_all,   val_loss_all,   val_ppl_all   = [], [], []

for path in ckpt_paths:
    event_paths = get_all_event_paths(path)
    print(f"\nFound {len(event_paths)} event files in {path}")

    # dict for step→value (last-write-wins on duplicates)
    train_loss_dict = {}
    val_loss_dict   = {}
    val_ppl_dict    = {}

    for event_path in event_paths:
        try:
            ea = event_accumulator.EventAccumulator(event_path)
            ea.Reload()

            train_loss_events = ea.Scalars("train/lm_loss")
            val_loss_events   = ea.Scalars("validation/lm_loss")
            val_ppl_events    = ea.Scalars("validation/lm_loss_ppl")

            # Overwrite repeated step entries with latest
            for e in train_loss_events:
                train_loss_dict[e.step] = e.value
            for e in val_loss_events:
                val_loss_dict[e.step] = e.value
            for e in val_ppl_events:
                val_ppl_dict[e.step] = e.value

        except Exception as ex:
            print(f"Error loading {event_path}: {ex}")
            continue

    if not train_loss_dict:
        print(f"No valid training data found in {path}. Skipping.")
        continue

    # Sort training steps
    train_steps_sorted = sorted(train_loss_dict.keys())
    train_loss_sorted  = [train_loss_dict[s] for s in train_steps_sorted]
    train_ppl_sorted   = [np.exp(v) for v in train_loss_sorted]

    # Sort validation steps
    val_steps_sorted = sorted(val_loss_dict.keys())
    val_loss_sorted  = [val_loss_dict[s] for s in val_steps_sorted]
    val_ppl_sorted   = [val_ppl_dict[s] for s in val_steps_sorted] if val_ppl_dict else []

    # Clip out early training steps with high PPL
    start_idx = next((i for i, v in enumerate(train_ppl_sorted) if v < ppl_start_threshold), 0)
    train_steps = train_steps_sorted[start_idx:]
    train_loss  = train_loss_sorted[start_idx:]
    train_ppl   = train_ppl_sorted[start_idx:]

    # Smooth train
    train_loss = smooth(train_loss, smoothing_train)
    train_ppl  = smooth(train_ppl,  smoothing_train)

    # Convert training steps to tokens
    train_steps = steps_to_tokens(train_steps)

    # For validation, skip early region if val_ppl exists
    if val_ppl_sorted:
        start_val_idx = next((i for i, v in enumerate(val_ppl_sorted) if v < ppl_start_threshold), 0)
    else:
        start_val_idx = 0
    val_steps = val_steps_sorted[start_val_idx:]
    val_loss  = val_loss_sorted[start_val_idx:]
    val_ppl   = val_ppl_sorted[start_val_idx:] if val_ppl_sorted else []
    val_loss  = smooth(val_loss, smoothing_val)
    val_ppl   = smooth(val_ppl,  smoothing_val)
    val_steps = steps_to_tokens(val_steps)

    # Store in final arrays
    train_steps_all.append(train_steps)
    train_loss_all.append(train_loss)
    train_ppl_all.append(train_ppl)

    val_steps_all.append(val_steps)
    val_loss_all.append(val_loss)
    val_ppl_all.append(val_ppl)

print("\nAll runs aggregated.")

##############################################################################
# 2) FIND THE MINIMUM FINAL STEP ACROSS ALL RUNS & CLIP
##############################################################################

# We'll look at the last training step for each run,
# take the minimum of those "final steps", call it `global_end_step`,
# and clip each run so we only keep data points up to that step.

# 2A) For each run, find final step
final_train_steps = []
for steps in train_steps_all:
    if len(steps) == 0:
        final_train_steps.append(0.0)
    else:
        final_train_steps.append(steps[-1])  # last step in tokens

# 2B) global_end_step = minimum of these final steps
global_end_step = min(final_train_steps)
print(f"Global end step = {global_end_step:.2f} (tokens)")

# 2C) Clip all runs to that end step
def clip_run_to_step(x_vals, y_vals, end_step):
    """
    Keep only the points where x_vals[i] <= end_step.
    x_vals and y_vals are same length, sorted in ascending order of x_vals.
    """
    clipped_x, clipped_y = [], []
    for (xx, yy) in zip(x_vals, y_vals):
        if xx <= end_step:
            clipped_x.append(xx)
            clipped_y.append(yy)
        else:
            break
    return clipped_x, clipped_y

num_runs = len(train_steps_all)
for i in range(num_runs):
    # Train
    old_x = train_steps_all[i]
    old_loss = train_loss_all[i]
    new_x, new_loss = clip_run_to_step(old_x, old_loss, global_end_step)
    train_steps_all[i] = new_x
    train_loss_all[i]  = new_loss

    old_ppl = train_ppl_all[i]
    _, new_ppl = clip_run_to_step(old_x, old_ppl, global_end_step)
    train_ppl_all[i] = new_ppl

    # Validation
    old_valx = val_steps_all[i]
    old_vloss= val_loss_all[i]
    new_valx, new_vloss = clip_run_to_step(old_valx, old_vloss, global_end_step)
    val_steps_all[i] = new_valx
    val_loss_all[i]  = new_vloss

    old_vppl = val_ppl_all[i]
    _, new_vppl = clip_run_to_step(old_valx, old_vppl, global_end_step)
    val_ppl_all[i] = new_vppl




Found 3 event files in /net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-lora/ad16eaf0ab491faf8f7dff350be0c984/tensorboard

Found 1 event files in /net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-base-moe/0e276af611df1ee7b123423ddae1e8e3/tensorboard

All runs aggregated.
Global end step = 7.43 (tokens)


In [9]:
# --- Plotting ---
fig, ax1 = plt.subplots(figsize=(7, 5))

# Train Loss
for i in range(len(ckpt_paths)):    
    ax1.plot(train_steps_all[i], train_loss_all[i], label=legend_keys[i])
ax1.set_title("Train Loss")
ax1.set_xlabel("Tokens (B)")
ax1.set_ylabel("Loss")


ax1.grid(True)
ax1.legend()

# Save Train Loss plot
train_loss_save_path = save_path.replace(".pdf", "_train_loss.pdf")
plt.savefig(train_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax2 = plt.subplots(figsize=(7, 5))

# Validation Loss
for i in range(len(ckpt_paths)):
    ax2.plot(val_steps_all[i], val_loss_all[i], label=legend_keys[i])
ax2.set_title("Validation Loss")
ax2.set_xlabel("Tokens (B)")
ax2.set_ylabel("Loss")
ax2.grid(True)
ax2.legend()

# Save Validation Loss plot
val_loss_save_path = save_path.replace(".pdf", "_val_loss.pdf")
plt.savefig(val_loss_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_loss_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax3 = plt.subplots(figsize=(7, 5))

# Train PPL
for i in range(len(ckpt_paths)):
    ax3.plot(train_steps_all[i], train_ppl_all[i], label=legend_keys[i])
ax3.set_title("Train Perplexity")
ax3.set_xlabel("Tokens (B)")
ax3.set_ylabel("PPL")
ax3.grid(True)
ax3.legend()

# Save Train PPL plot
train_ppl_save_path = save_path.replace(".pdf", "_train_ppl.pdf")
plt.savefig(train_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {train_ppl_save_path}")
plt.close(fig)

# --- Plotting ---
fig, ax4 = plt.subplots(figsize=(7, 5))

# Validation PPL
for i in range(len(ckpt_paths)):
    ax4.plot(val_steps_all[i], val_ppl_all[i], label=legend_keys[i])
ax4.set_title("Validation Perplexity")
ax4.set_xlabel("Tokens (B)")
ax4.set_ylabel("PPL")
ax4.grid(True)
ax4.legend()

# Save Validation PPL plot
val_ppl_save_path = save_path.replace(".pdf", "_val_ppl.pdf")
plt.savefig(val_ppl_save_path, dpi=300, bbox_inches='tight')
print(f"Saved plot to: {val_ppl_save_path}")
plt.close(fig)

Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/1c1-l-16-r8_1c1_train_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/1c1-l-16-r8_1c1_val_loss.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/1c1-l-16-r8_1c1_train_ppl.pdf
Saved plot to: /home/zsarwar/Projects/gpt_neox/gpt-neox/code/plots/1c1-l-16-r8_1c1_val_ppl.pdf


# 1c1 LORA ENDS


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator



# ckpt_paths = [
#     "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12/b001-9aae413635a59a1bd748912a1d6a8077/tensorboard",
#     # "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12/j002-f8d4350954da3c455cfb58e12936d732/tensorboard",
#     "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12/w002-300bc04cda9f137ea77a2da53d51d3f4/tensorboard",
#     # "/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12/k001-e219b0857385c22aca203fa5ca4c6110_2/tensorboard"
# ]
# legend_keys = [
#     "1c1 + l-16-r-8",
#     # "1c1 + l-8_r-4",
#     "1c1",
#     # "1c1 + l-16-r-4",

# ]


# # === Config ===
# ckpt_paths = [
#     "/net/scratch/zsarwar/exps/GPT_experts-8-topk-1-layers12-heads-12/k001-38abff837ec0d231d3e9eb56c9cdb6ac/tensorboard/",
#     # "/net/scratch/zsarwar/exps/GPT_experts-8-topk-2-layers12-heads-12/k001-bf438c8582fa76d72c2527fa61802b74/tensorboard",
#     "/net/scratch/zsarwar/exps/GPT_experts-7-topk-1-layers12-heads-12/e002-af3b953a6664974775edb480246a64fc/tensorboard",
#     # "/net/scratch/zsarwar/exps/GPT_experts-7-topk-1-layers12-heads-12/b003-82d0e08a0c26696f7991aaa0671ae599/tensorboard"
# ]
# legend_keys = [
#     "8c1",
#     # "8c2",
#     "7c1+Lora8c1",
#     # "7c1",

# ]


# # === Config ===
# ckpt_paths = [
# "/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12/w001-446d5e9e78a690921a149c65320e1eea/tensorboard/",
# "/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12/w001-ec38fe523db7eb59419fcd573e308e8d/tensorboard/"
# ]
# legend_keys = [
#     "4c1",
#     "4c1 + l-16-r-8"
# ]



# # === Config ===
# ckpt_paths = [
# "/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-hetro/e3d7580a3c9ad108011ed2eb4321473f/tensorboard",
# "/net/scratch/zsarwar/exps/GPT_experts-4-topk-1-layers12-heads-12-base-moe/14a61de6f166d8f3ccae64a958e06558/tensorboard",
# "/net/scratch/zsarwar/exps_t/GPT_experts-4-topk-1-layers12-heads-12/w001-446d5e9e78a690921a149c65320e1eea/tensorboard"
# ]
# legend_keys = [
#     "4c1-hetro",
#     "4c1 + param-matched",
#     "4c1"

# ]

ckpt_paths = [
"/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-lora/ad16eaf0ab491faf8f7dff350be0c984/tensorboard",
"/net/scratch/zsarwar/exps/GPT_experts-1-topk-1-layers12-heads-12-base-moe/0e276af611df1ee7b123423ddae1e8e3/tensorboard"
]
legend_keys = [
    "1c1-l-16-r8",
    "1c1",
    

]



key_x = '_'.join(legend_keys)
save_path = f"/home/zsarwar/Projects/neox/gpt-neox-base-moe/code/plots/{key_x}.pdf"  # Set to None to disable saving


def steps_to_tokens(steps):
    dp_size = 16
    batch_size = 36
    seq_len = 2048
    divv = 1e9
    mult = (batch_size * seq_len * dp_size) / divv
    tokens = [step * mult for step in steps]
    return tokens

ppl_start_threshold = 70
smoothing_train = 0.9
smoothing_val = 0.4

def smooth(vals, factor):
    if factor == 0:
        return vals
    smoothed = []
    last = vals[0]
    for v in vals:
        last = last * factor + (1 - factor) * v
        smoothed.append(last)
    return smoothed

# --- Storage ---
train_loss_all = []
val_loss_all = []
train_ppl_all = []
val_ppl_all = []
train_steps_all = []
val_steps_all = []

# --- Load runs ---
for path in ckpt_paths:
    ea = event_accumulator.EventAccumulator(path)
    ea.Reload()

    train_loss_events = ea.Scalars("train/lm_loss")
    val_loss_events = ea.Scalars("validation/lm_loss")
    val_ppl_events = ea.Scalars("validation/lm_loss_ppl")

    steps = [e.step for e in train_loss_events]
    # if "e3d7580a3c9ad108011ed2eb4321473f" in path:
        # train_loss = [(e.value - 0.05) for e in train_loss_events]
    # else:
    train_loss = [e.value for e in train_loss_events]
    train_ppl = [np.exp(v) for v in train_loss]

    start = next((i for i, v in enumerate(train_ppl) if v < ppl_start_threshold), 0)

    steps = steps_to_tokens(steps[start:])

    train_loss = smooth(train_loss[start:], smoothing_train)
    train_ppl = smooth(train_ppl[start:], smoothing_train)

    val_steps = steps_to_tokens([e.step for e in val_loss_events])
    val_loss = smooth([e.value for e in val_loss_events], smoothing_val)
    val_ppl = smooth([e.value for e in val_ppl_events], smoothing_val)



    train_steps_all.append(steps)
    val_steps_all.append(val_steps)
    train_loss_all.append(train_loss)
    val_loss_all.append(val_loss)
    train_ppl_all.append(train_ppl)
    val_ppl_all.append(val_ppl)



# --- Clip to shortest run for training plots ---
min_train_len = min(len(steps) for steps in train_steps_all)

for i in range(len(train_steps_all)):
    train_steps_all[i] = train_steps_all[i][:min_train_len]
    train_loss_all[i] = train_loss_all[i][:min_train_len]
    train_ppl_all[i] = train_ppl_all[i][:min_train_len]


# --- Clip to shortest run for valing plots ---
min_val_len = min(len(steps) for steps in val_steps_all)

for i in range(len(val_steps_all)):
    val_steps_all[i] = val_steps_all[i][:min_val_len]
    val_loss_all[i] = val_loss_all[i][:min_val_len]
    val_ppl_all[i] = val_ppl_all[i][:min_val_len]







# --- Plotting ---
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
(ax1, ax2), (ax3, ax4) = axes

# Train Loss
for i in range(len(ckpt_paths)):
    ax1.plot(train_steps_all[i], train_loss_all[i], label=legend_keys[i])
ax1.set_title("Train Loss")
ax1.set_xlabel("Tokens (B)")
ax1.set_ylabel("Loss")
ax1.grid(True)
ax1.legend()

# Validation Loss
for i in range(len(ckpt_paths)):
    ax2.plot(val_steps_all[i], val_loss_all[i], label=legend_keys[i])
ax2.set_title("Validation Loss")
ax2.set_xlabel("Tokens (B)")
ax2.set_ylabel("Loss")
ax2.grid(True)
ax2.legend()

# Train PPL
for i in range(len(ckpt_paths)):
    ax3.plot(train_steps_all[i], train_ppl_all[i], label=legend_keys[i])
ax3.set_title("Train Perplexity")
ax3.set_xlabel("Tokens (B)")
ax3.set_ylabel("PPL")
ax3.grid(True)
ax3.legend()

# Validation PPL
for i in range(len(ckpt_paths)):
    ax4.plot(val_steps_all[i], val_ppl_all[i], label=legend_keys[i])
ax4.set_title("Validation Perplexity")
ax4.set_xlabel("Tokens (B)")
ax4.set_ylabel("PPL")
ax4.grid(True)
ax4.legend()

plt.tight_layout()



# --- Save as PDF if specified ---
if save_path:
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Saved plot to: {save_path}")


plt.show()